In [1]:
import numpy as np
import cobra
import sbtab
from equilibrator_api import ComponentContribution, Reaction, Pathway, Q_
from urllib.request import urlopen
from sbtab import sbml2sbtab
from importlib import reload
import warnings
warnings.filterwarnings('ignore')

Fragments already downloaded
Fragments already downloaded


In [2]:
m=cobra.io.load_matlab_model('./model/GSmodel_Ecoli.mat')

In [3]:
import json
#LOAD dG CONSTRAINTS
# use fp ./dG_I_0mM.json, ./dG_I_100mM.json or ./dG_I_250mM.json for delta Gs at 0, 100 or 250 mM, respectively.
with open('./params/dG_I_250mM.json','r') as f: 
    dG_dict=json.load(f)

In [4]:
import fix_dicts
import importlib
importlib.reload(fix_dicts)
from fix_dicts import correct_reversibilities
import mdf_LP
from mdf_LP import run_mdf_LP
importlib.reload(mdf_LP)

<module 'mdf_LP' from '/home/mbzron/PycharmProjects/metabolites_LP/mdf_LP.py'>

In [5]:
#PERFORM ANALYSIS
#define analysis_type as FBA or TFA.

analysis_type='TFA' #i.e. the analysis from which the flux distribution has been derived

# fp='./'+analysis_type+'_fluxes_table.txt' #fluxes
fp='./fluxes/'+analysis_type+'_MDF_2019_10_06.txt' #fluxes
f=open(fp,'r')
flines=f.readlines()

#THIS LINE NEEDS TO CHANGE FOR LATEST ITERATION OF DATA
#ALSO NEED TO ADJUST 'CORRECT_REVERSIBILITIES' TO IGNORE SMALL '-E' FLUXES
#run_list=['Run '+str(x+1) for x in range(64)] #list of 'run' headers in spreadsheet, 1-64
run_list=[analysis_type+'_fluxes'+str(x+2) for x in range(64)] #list of 'run' headers in spreadsheet, 1-64, #for updated FBA/TFA dict

mdf_dict={'mdf':{}} #store results here

for run in run_list:
    
    corrected_dG_dict,flux_dict=correct_reversibilities(run,dG_dict,flines,analysis_type) #change sign of dG for reverse reactions
    c=run_mdf_LP(m,flux_dict,corrected_dG_dict) #run mdf analysis
    
    #store results in dictionary
    if c:
        mdf_dict['mdf'][run]=c['mdf']
        for metabolite in c['metabolites'].keys():
            if metabolite not in mdf_dict.keys():
                mdf_dict[metabolite]={run:c['metabolites'][metabolite]}
            elif metabolite in mdf_dict.keys():
                mdf_dict[metabolite][run]=c['metabolites'][metabolite]
    else:
        mdf_dict['mdf'][run]='INFEASIBLE'
    
f.close()

mdf: -0.0023 kJ/mol
B: 0.9112845860199741
mdf: -0.0023 kJ/mol
B: 0.9112845995323369
mdf: -0.0025 kJ/mol
B: 1.0275016462694566
mdf: -0.0025 kJ/mol
B: 1.0275016841303568
mdf: -0.0023 kJ/mol
B: 0.9112845860199741
mdf: -0.0023 kJ/mol
B: 0.9112845995323369
mdf: -0.0025 kJ/mol
B: 1.0275016462694566
mdf: -0.0025 kJ/mol
B: 1.0275016841303568
mdf: -0.0023 kJ/mol
B: 0.9112845860199741
mdf: -0.0023 kJ/mol
B: 0.9112845995323369
mdf: -0.0025 kJ/mol
B: 1.0275030175062057
mdf: -0.0025 kJ/mol
B: 1.0275010424804805
mdf: -0.0023 kJ/mol
B: 0.9112845860199741
mdf: -0.0023 kJ/mol
B: 0.9112845995323369
mdf: -0.0023 kJ/mol
B: 0.9112836121462351
mdf: -0.0023 kJ/mol
B: 0.9112846113323485
mdf: -0.0023 kJ/mol
B: 0.9112845860199741
mdf: -0.0023 kJ/mol
B: 0.9112845995323369
mdf: -0.0025 kJ/mol
B: 1.0274989010183617
mdf: -0.0023 kJ/mol
B: 0.9112845980803286
mdf: -0.0023 kJ/mol
B: 0.9112845860199741
mdf: -0.0023 kJ/mol
B: 0.9112845995323369
mdf: -0.0025 kJ/mol
B: 1.0274989010183617
mdf: -0.0023 kJ/mol
B: 0.911284598

In [6]:
#write results file
fname='./results/TFA_results_250_19-10-2019.csv' #CHOOSE FILE NAME

run_list=[analysis_type+'_fluxes'+str(x+2) for x in range(64)] #list of 'run' headers in spreadsheet, 1-64, #for updated FBA/TFA dict

with open(fname,'w+') as f:
    f.write('\t'.join(['metabolite ID']+run_list+['\n']))
    for metabolite in list(mdf_dict.keys()):
        row=[metabolite]
        for run in run_list:
            if run in mdf_dict[metabolite].keys():
                row.append(mdf_dict[metabolite][run])
            else:
                row.append(0)
        f.write('\t'.join([str(x) for x in row+['\n']]))